In [3]:
%%sh
wget -N https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip

--2023-12-18 17:38:42--  https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip
Resolving sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com (sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com)... 52.92.149.146, 3.5.87.65, 52.218.176.33, ...
Connecting to sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com (sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com)|52.92.149.146|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 432828 (423K) [application/zip]
Saving to: ‘bank-additional.zip’

     0K .......... .......... .......... .......... .......... 11%  772K 0s
    50K .......... .......... .......... .......... .......... 23%  387K 1s
   100K .......... .......... .......... .......... .......... 35%  775K 0s
   150K .......... .......... .......... .......... .......... 47% 84.6M 0s
   200K .......... .......... .......... .......... .......... 59%  163M 0s
   250K .......

In [7]:
!unzip -o bank-additional.zip

Archive:  bank-additional.zip
   creating: bank-additional/
  inflating: bank-additional/bank-additional-names.txt  
  inflating: bank-additional/bank-additional.csv  
  inflating: bank-additional/bank-additional-full.csv  


In [9]:
import pandas as pd
data = pd.read_csv('bank-additional/bank-additional-full.csv')
print(data.shape)
data[:5]

(41188, 21)


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [11]:
import sagemaker
prefix = 'sagemaker/DEMO-smprocessing/input'
input_data = sagemaker.Session().upload_data(path='bank-additional/bank-additional-full.csv'
                                             ,key_prefix=prefix)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [12]:
from sagemaker.sklearn.processing import SKLearnProcessor

role = sagemaker.get_execution_role()
sklearn_processor = SKLearnProcessor(framework_version='0.20.0',
                                     role=role,
                                     instance_type='ml.m5.xlarge',
                                     instance_count=1)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [13]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

sklearn_processor.run(code='preprocessing.py',
                      inputs=[ProcessingInput(source=input_data,
                                              destination='/opt/ml/processing/input')],
                      outputs=[ProcessingOutput(output_name='train_data',
                                                source='/opt/ml/processing/train'),
                               ProcessingOutput(output_name='test_data',
                                                source='/opt/ml/processing/test')]
                     )

INFO:sagemaker:Creating processing-job with name sagemaker-scikit-learn-2023-12-18-17-45-44-178


............................/miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/miniconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'
Reading input data from /opt/ml/processing/input/bank-additional-full.csv
Positive samples: 4639
Negative samples: 36537
Ratio: 7.88
Running preprocessing and feature engineering transformations
Train data shape after preprocessing: (32940, 58)
Test data shape after preprocessing: (8236, 58)
Saving training features to /opt/ml/processing/train/train_features.csv
Saving test features to /opt/ml/processing/test/test_features.csv
Saving training labels to /opt/ml/processing/train/train_labels.c

In [14]:
preprocessing_job_description = sklearn_processor.jobs[-1].describe()

output_config = preprocessing_job_description['ProcessingOutputConfig']
for output in output_config['Outputs']:
    print(output['S3Output']['S3Uri'])

s3://sagemaker-us-east-1-466175814801/sagemaker-scikit-learn-2023-12-18-17-45-44-178/output/train_data
s3://sagemaker-us-east-1-466175814801/sagemaker-scikit-learn-2023-12-18-17-45-44-178/output/test_data


In [15]:
# 2. Training a model using SageMaker SDK with built-in algorithms

In [19]:
import pandas as pd

dataset = pd.read_csv('housing.csv')
print(dataset.shape)
dataset[:5]

(506, 13)


,crim,zn,indus,chas,nox,age,rm,dis,rad,tax,ptratio,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,5.33,36.2


In [20]:
dataset = pd.concat([dataset['medv'], dataset.drop(['medv'], axis=1)],axis=1)

In [21]:
from sklearn.model_selection import train_test_split

training_dataset, validation_dataset = train_test_split(dataset,test_size=0.1)

print(training_dataset.shape)
print(validation_dataset.shape)


(455, 13)
(51, 13)


In [22]:
training_dataset.to_csv('training_dataset.csv', index=False, header=False)

validation_dataset.to_csv('validation_dataset.csv', index=False, header=False)

In [23]:
import sagemaker

print(sagemaker.__version__)

2.199.0


In [24]:
sess = sagemaker.Session()
bucket = sess.default_bucket()
print(bucket)
prefix = 'boston-housing'
training_data_path = sess.upload_data(path='training_dataset.csv', 
key_prefix=prefix + '/input/training')
validation_data_path = sess.upload_data(path='validation_dataset.csv', 
key_prefix=prefix + '/input/validation')
print(training_data_path)
print(validation_data_path)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker-us-east-1-466175814801
s3://sagemaker-us-east-1-466175814801/boston-housing/input/training/training_dataset.csv
s3://sagemaker-us-east-1-466175814801/boston-housing/input/validation/validation_dataset.csv


In [25]:
import boto3
from sagemaker import image_uris

region = boto3.Session().region_name 
print(region)
container = image_uris.retrieve('linear-learner', region)
print(container)

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: 1.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


us-east-1
382416733822.dkr.ecr.us-east-1.amazonaws.com/linear-learner:1


In [26]:
from sagemaker.estimator import Estimator
role = sagemaker.get_execution_role() 

ll_estimator = Estimator(container,
 role=role, 
 instance_count=1,
 instance_type='ml.m5.large',
 output_path='s3://{}/{}/output'.format(bucket, prefix)
)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [27]:
ll_estimator.set_hyperparameters(predictor_type='regressor',
                                 mini_batch_size=32)


In [28]:
training_data_channel = sagemaker.TrainingInput(s3_data=training_data_path,content_type='text/csv')
validation_data_channel = sagemaker.TrainingInput(s3_data=validation_data_path,content_type='text/csv')

ll_data = {'train': training_data_channel, 'validation': validation_data_channel}

In [29]:
ll_estimator.fit(ll_data)

INFO:sagemaker:Creating training-job with name: linear-learner-2023-12-18-18-03-12-593


2023-12-18 18:03:12 Starting - Starting the training job...
2023-12-18 18:03:29 Starting - Preparing the instances for training.........
2023-12-18 18:04:57 Downloading - Downloading input data......
2023-12-18 18:05:47 Downloading - Downloading the training image......
2023-12-18 18:07:13 Training - Training image download completed. Training in progress.
2023-12-18 18:07:13 Uploading - Uploading generated training modelDocker entrypoint called with argument(s): train
Running default environment configuration script
[12/18/2023 18:07:03 INFO 140556005410624] Reading default configuration from /opt/amazon/lib/python3.8/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07

In [30]:
from time import strftime, gmtime

timestamp = strftime('%d-%H-%M-%S', gmtime())
endpoint_name = 'linear-learner-demo-'+timestamp
print(endpoint_name)

linear-learner-demo-18-18-11-11


In [31]:
ll_predictor = ll_estimator.deploy(endpoint_name=endpoint_name,
                                   initial_instance_count=1,
                                   instance_type='ml.t2.medium')

INFO:sagemaker:Creating model with name: linear-learner-2023-12-18-18-11-39-101
INFO:sagemaker:Creating endpoint-config with name linear-learner-demo-18-18-11-11
INFO:sagemaker:Creating endpoint with name linear-learner-demo-18-18-11-11


-------------------!

In [32]:
test_sample = '0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,4.98'
#ll_predictor.content_type = 'text/csv'
ll_predictor.serializer = sagemaker.serializers.CSVSerializer()
ll_predictor.deserializer = sagemaker.deserializers.CSVDeserializer()
response = ll_predictor.predict(test_sample)
print(response)

[['30.397703170776367']]


In [33]:
ll_predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: linear-learner-demo-18-18-11-11
INFO:sagemaker:Deleting endpoint with name: linear-learner-demo-18-18-11-11
